In [1]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
from datetime import datetime
import arcpy

arcpy.env.overwriteOutput = True

In [2]:
sheet_name = 'CONTROL_DE_CAMBIOS'
sheet_id = '1n_t9qtwLeIklH5UFoH9iDPiAa6uN4HZeR1-atsKTYeU'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

control_cambios_esig_original = pd.read_csv(url)
columnas_seleccion = ['terreno_codigo', 
    'Nombre', 
    'EDICION',
    'POSIBLE SERVIDUMBRE', 
    'FECHA EDITOR',
    'OBSERVACIONES EDITOR',
    'REVISADO',
    'FECHA REVISOR',
    'OBSERVACIONES REVISOR',
    'ESTADO DE EDICION']

control_cambios_esig_filtro = control_cambios_esig_original[columnas_seleccion]

In [3]:
control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'REQUIERE VISITA DE CAMPO', 'EDICION'] = 'Requiere visita campo'
control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'SI, (EDICION GEOGRAFICA)', 'EDICION'] = 'Con edición geo'
control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'NO, (NO REQUIERE AJUSTE GEOGRAFICO)', 'EDICION'] = 'No requiere ajuste geo'

In [4]:
ruta_filebd = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
ruta_shp = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoSIG\1.Geo_Gestion_Cambios\zShps"

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'
SHP = '_Gestion_Cambios_SIG.shp'

fecha_actual = datetime.now()
fecha_jpeg = str(fecha_actual.strftime("%Y-%m-%d"))

nombre_shp = str(fecha_jpeg) + SHP

capa_terrenos_rurales = os.path.join(ruta_filebd, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(ruta_filebd, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)
df_terreno_rural.head()
# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
                        'CODIGO_ANTERIOR',
                        'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

In [5]:

# ** Terrenos Gestionados SIG Espacializados
df_terrenos_gestion_sig = pd.merge(df_terrenos, control_cambios_esig_filtro, left_on='CODIGO', right_on='terreno_codigo', how='inner')

In [6]:
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'terrenos_con_control_de_cambios_esig'.upper()}"
ruta_salida_terrenos_contro_cambios = os.path.join(ruta_filebd, NOMBRE_CAPA_SALIDA)

df_terrenos_gestion_sig.spatial.to_featureclass(location=ruta_salida_terrenos_contro_cambios)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

Se exporta la capa Analitica_UT_Consolidada\TERRENOS_CON_CONTROL_DE_CAMBIOS_ESIG


In [7]:

ruta_salida_shp = os.path.join(ruta_shp, nombre_shp)
df_terrenos_gestion_sig.spatial.to_featureclass(location=ruta_salida_shp)
print(f"Se exporta la capa {nombre_shp}")

cannot add field: 'observaciones_revisor'
Se exporta la capa 2024-04-30_Gestion_Cambios_SIG.shp
